In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import datetime
from sklearn.utils import shuffle
import tensorboard

Source and Shuffle

In [ ]:
inputs_set = np.load("./Titanic-misc/train.npz")

In [ ]:
inputs_unshuffled = inputs_set["inputs"]
targets_unshuffled = inputs_set["outputs"]


inputs_shuffled, targets_shuffled = shuffle(inputs_unshuffled, targets_unshuffled)

In [ ]:
train_size = np.int64(targets_shuffled.size * 0.8)

train_inputs = inputs_shuffled[0:train_size,:]
train_targets = targets_shuffled[0:train_size]

validation_inputs = inputs_shuffled[train_size:,:]
validation_targets = targets_shuffled[train_size:]


# Model

HP HyperParams

In [ ]:
HP_input_size = inputs_shuffled[0].size
HP_hidden_size = 14
HP_output_size = 2

HP_epochs = 100
BATCH_PER_EPOCH = 10 # batch per 1 Epoch

HP_optimizer = tf.keras.optimizers.Adam() # learning rate adaptive schedule + momentum
HP_loss = tf.keras.losses.sparse_categorical_crossentropy

# TensorBoard
log_dir = "./Titanic-misc/logs" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Early Stopping
earlyStopping = tf.keras.callbacks.EarlyStopping(patience=10)

Model structure

In [ ]:
model = tf.keras.Sequential([
    # Flatter layer skipped because all preprocessing was done earlier
    tf.keras.layers.Dense(HP_hidden_size,activation='relu'),
    tf.keras.layers.Dense(HP_hidden_size,activation='relu'),
    tf.keras.layers.Dense(HP_output_size,activation='softmax')
])

Compile

In [ ]:
model.compile(optimizer=HP_optimizer, loss=HP_loss, metrics=['accuracy'])

In [ ]:
%load_ext tensorboard

Fit the model

In [ ]:
model.fit(train_inputs, train_targets,
          epochs=HP_epochs,
          batch_size=BATCH_PER_EPOCH,
          validation_data=[validation_inputs,validation_targets],
          callbacks=[tensorboard_callback,earlyStopping],
          verbose = 1)

In [ ]:
%tensorboard --logdir "./Titanic-misc"

# Predict

In [ ]:
test_set = np.load("./Titanic-misc/test.npz")
test_inputs = test_set["inputs"]

In [ ]:
test_predict = model.predict_on_batch(test_inputs)

In [ ]:
results = pd.read_csv("./Titanic-misc/test.csv")
results = pd.concat([results,pd.DataFrame(test_predict, columns=["Not Survived","Survived"])], axis = 1)

In [ ]:
def survival(x):
    if x<0.5:
        return 0
    else:
        return 1
    

export = results[["PassengerId","Survived"]]
export["Survived"]=export["Survived"].apply(survival)

In [ ]:
export.to_csv("./Titanic-misc/result_TF_"+np.str(pd.Timestamp.now())+".csv", index=False)